# Amazon Recommendation System Based On Preview

In [3]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

In [2]:
import kagglehub
import os
import shutil
# Download latest version
path = kagglehub.dataset_download("karkavelrajaj/amazon-sales-dataset")
data_dir = "./datasets"

os.makedirs(data_dir, exist_ok=True)
shutil.copytree(path, data_dir, dirs_exist_ok=True)

print("Dataset đã được lưu tại:", os.path.abspath(data_dir))

Dataset đã được lưu tại: /home/dikhangcshcmut/Code/code_for_fun/Project/RecommendationProduct/AmazonRecommendationBasedOnPreview/datasets
